In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("raw_data/single_node.csv").rename(columns={"world_size": "World"})

In [ ]:
# Split by backend
gloo = df[df.backend == "gloo"].pivot(
    index="tensor_size", columns="World", values="mean"
)

nccl = df[df.backend == "nccl"].pivot(
    index="tensor_size", columns="World", values="mean"
)

# Combine into desired format
report = gloo.combine(
    nccl,
    lambda a, b: a.combine(
         b,
        lambda x, y: (
            f"{x*1000:.2f} / {y*1000:.2f}"
        )
    )
)

report.index = report.index.str.replace("MiB", "MB").str.replace("GiB", "GB")
order = ["1MB", "10MB", "100MB", "1GB"]
report = report.reindex(order)

In [ ]:
report

In [ ]:
tex = report.to_latex(index=True, index_names=False, caption=f"Mean latency: GLoo / NCCL (ms)", escape=False)
tex = tex.replace(r"\begin{table}", r"\begin{table}[H]\centering")
print(tex)